<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/CreateGSPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение данных


In [ ]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

# worksheet = gc.open('План работ Договор Адамант').sheet1
worksheet = gc.open('План работ Договор Адамант')

# get_all_values gives a list of rows.
# rows = worksheet.get_all_values()
# print(rows)

# # Convert to a DataFrame and render.
# pd.DataFrame.from_records(rows)

* данный финансового плана импортируются из АСС, папка соответсвующего договора TODO
* объем по контракту необходимо формировать из бим-модели, либо подгружать из РД, через процедуру проверки по ППР TODO

In [ ]:
import numpy as np
fin = worksheet.worksheet("FINANCE PLAN")
fin = fin.get_all_values()
fin = np.array(fin)
fin =  pd.DataFrame(fin[5:, :7])
fin

In [ ]:
fin.rename(columns = {0:'id', 
                       1:'WORK',2:'START', 3:'FINISH', 4:'PRICE for ONE', 5:'UNITS OF MEASUREMENT',6:'VALUE CONTRACT'}, 
            inplace = True)

In [ ]:
fin = fin.set_index("id")
fin = fin[fin.index > '']

Приводим строковые значения к числовым 

In [ ]:
fin['PRICE for ONE'] = fin['PRICE for ONE'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
fin['VALUE CONTRACT'] = fin['VALUE CONTRACT'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))

Приводим значения дат в формат date

In [ ]:
fin['START'] = pd.to_datetime(fin['START'])   
fin['FINISH']= pd.to_datetime(fin['FINISH'])   

Завершаем предобработку таблицы - заменяем все значения Nan на "0"

In [ ]:
fin = fin.fillna(0)

# Формирование листа FINANCE

Задача: на основе данных из условий договора и данных по планируемому и фактическому выполнению (сейчас данные по факту просто берем из таблицы ПЛАН)

* расчет общей стоимости по контракту
* формирование плановой стоимости работ на соответсвующую дату: за период
* формирование плановой стоимости работ на соответсвующую дату: накопительная сумма
* формирование фактической стоимости работ на соответсвующую дату: за период
* формирование фактической стоимости работ на соответсвующую дату: накопительная сумма
* внесение корректировок и обновлений при формировании ДС

❗ сравнение с нашей базой расценок 




In [ ]:
fin['TOTAL PRICE'] = fin['PRICE for ONE'] * fin['VALUE CONTRACT']

# Предобработка листа ГРАФИК РАБОТ

Подключаем лист ГРАФИК РАБОТ

In [482]:
graf = worksheet.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])

In [ ]:
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'id', 1: 'WORK'}, inplace=True) 
graf_name = graf_name[graf_name['id'] != 0]
graf_name = pd.DataFrame(graf_name['WORK'].values, index=graf_name['id'], columns=['WORK'])

Предобработка данных в таблице:
* приведение данных в необходимый формат
* корректировка наименования столбцов и индексов

In [484]:
graf.drop(graf.index[1:10], axis=1, inplace=True)

Формируем два списка для мультииндексов

In [485]:
newColumnsIndex = [pd.to_datetime(pd.Series(graf.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf.iloc[1][1:].values]

In [486]:
graf.rename(columns={0: 'id'}, inplace=True)

In [487]:
graf = graf.set_index("id")

In [488]:
graf = pd.DataFrame(graf.values, index=graf.index, columns=newColumnsIndex)

In [489]:
graf.drop(graf.index[:2], axis=0, inplace=True)

In [490]:
for i in graf.columns:
    graf[i] = graf[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))

In [ ]:
graf.fillna(0, inplace=True)
graf.sort_index(axis=0, level=1, ascending=True)

# ПЛАН/ФАКТ
* выполненных работ
* стоимости выполненных работ
* формирование КС


In [492]:
idx = pd.IndexSlice

In [497]:
plan = graf.loc[:, idx['2022-12-12':'2022-12-19','план']].sum(axis=1).values
total_plan = graf.loc[:, idx[:,'план']].sum(axis=1).values
plan = pd.Series(plan, index=graf.index, name='PLAN_WEEK')
total_plan = pd.Series(total_plan, index=graf.index, name='TOTAL_PLAN')

In [523]:
plan_next = graf.loc[:, idx['2022-12-20':'2022-12-27','план']].sum(axis=1).values
plan_next = pd.Series(plan_next, index=graf.index, name='PLAN_NEXT_WEEK')

In [498]:
fakt = graf.loc[:, idx['2022-12-12':'2022-12-19','факт']].sum(axis=1).values
fakt = pd.Series(fakt, index=graf.index, name='FAKT_WEEK')
total_fakt = graf.loc[:, idx[:,'факт']].sum(axis=1).values
total_fakt = pd.Series(total_fakt, index=graf.index, name='TOTAL_FAKT')

In [528]:
fakt_next = graf.loc[:, idx['2022-12-20':'2022-12-27','факт']].sum(axis=1).values
fakt_next = pd.Series(fakt_next, index=graf.index, name='FAKT_NEXT_WEEK')

In [529]:
planFakt = pd.concat([graf_name,total_plan, total_fakt,plan,fakt,plan_next,fakt_next], axis=1)

In [ ]:
planFakt = planFakt[planFakt['PLAN_WEEK'] != 0]

# Лист PLAN_WEEK

TODO => chartWEEK, ед измерения, менять даты по дням, chart оплтат

In [537]:
worksheet.add_worksheet('PLAN_WEEK',planFakt.shape[0], planFakt.shape[1])

<Worksheet 'PLAN_WEEK' id:1094968404>

In [544]:
worksheet.worksheet('PLAN_WEEK').clear()

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'clearedRange': 'PLAN_WEEK!A1:G61'}

In [546]:
worksheet.worksheet('PLAN_WEEK').append_rows([planFakt.columns.values.tolist()]+planFakt.values.tolist())

{'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
 'updates': {'spreadsheetId': '1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc',
  'updatedRange': 'PLAN_WEEK!A1:G31',
  'updatedRows': 31,
  'updatedColumns': 7,
  'updatedCells': 217}}

In [457]:
dir(worksheet.worksheet('PLAN_WEEK'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_finder',
 '_properties',
 'acell',
 'add_cols',
 'add_rows',
 'append_row',
 'append_rows',
 'batch_get',
 'batch_update',
 'cell',
 'clear',
 'clear_basic_filter',
 'client',
 'col_count',
 'col_values',
 'delete_row',
 'delete_rows',
 'duplicate',
 'export',
 'find',
 'findall',
 'format',
 'freeze',
 'frozen_col_count',
 'frozen_row_count',
 'get',
 'get_all_records',
 'get_all_values',
 'id',
 'insert_row',
 'merge_cells',
 'range',
 'resize',
 'row_count',
 'row_values',
 'set_basic_filter',
 'sort',
 'spreadsheet',
 'title',
 'update',
 'update_acell',
 'update_cell',
 'update_cells',
 'update_index',
 'update_tit